# AI Judge pre-processing - skillborders

In [1]:
!pip3 install opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import numpy as np
import pandas as pd
import pickle
import cv2 # pip install opencv-python

# constanten

In [3]:
video_border_labels_path = '../data/df_video_border_labels.pkl'

video_folder = '../videos/'

video_names = [
    '20240201_atelier_001.mp4',
    '20240201_atelier_002.mp4',
    '20240201_atelier_003.mp4',
    '20240201_atelier_004.mp4',
    '20240201_atelier_005.mp4',
    '20240209_atelier_006.mp4',
    '20240209_atelier_007.mp4',
    '20240209_atelier_008.mp4',
]

train_videos = [ video_folder + trainvid for trainvid in video_names]

train_videos = [
    'videos/20240201_atelier_001.mp4',
    'videos/20240201_atelier_002.mp4',
    'videos/20240201_atelier_003.mp4',
    'videos/20240201_atelier_004.mp4',
    'videos/20240201_atelier_005.mp4',
    'videos/20240209_atelier_006.mp4',
    'videos/20240209_atelier_007.mp4',
    'videos/20240209_atelier_008.mp4',
]

# functies

In [4]:
def pickle_load_or_create(path, cols):
    if os.path.exists(path):
        with open(path, 'rb') as file:
            return pickle.load(file)
    else:
        return pd.DataFrame(columns=cols)

In [5]:
def get_random_frame(videonames, videofolder, framelabels):
    """
    videos: array of video_paths
    df_video_border_labels: panda dataframe ['path', 'frame', 'borderlabel']
        [ 0 : ground
          1 : heels of ground
          2 : air ]
    """
    vid_id = np.random.randint(0, len(videonames)-1)
    vid_name = videonames[vid_id]
    path = videofolder + vid_name
    print(path)
    cap = cv2.VideoCapture(path)
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # print(len(framelabels[framelabels['path'] == path]) / video_length) 
    
    frame_nr = np.random.randint(0, video_length-1)
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_nr)
    res, frame = cap.read()
    cap.release()
    # cv2.destroyAllWindows()

    return vid_name, frame_nr, frame

In [6]:
def show_frame(frame, scale=0.4):
    """
    Displays a single frame using OpenCV, waits for a key press, 
    and then closes the frame window.

    Args:
        frame (numpy.ndarray): The image frame to display.

    Returns:
        int: The ASCII value of the key pressed.
    """
    if scale != 1.0:
        frame = cv2.resize(frame, dsize=(0,0), fx=scale, fy=scale)
    # Display the frame using OpenCV
    cv2.imshow('Frame', frame)
    
    # Wait for a key press
    key = cv2.waitKey(0)

    return key

In [7]:
def match_label(key):
    match key:
        case 'n': 
            return 9
        case 'f':
            return 5
        case 'a' | '2':
            return 2
        case 'h' | 's' | 'm' | '1':
            return 1
        case 'g' | '0':
            return 0

def is_already_labeled(path, frame_nr, df_labels):
    return len(df_labels[(df_labels['path'] == path) & (df_labels['frame'] == frame_nr)]) > 0

def label_frames(df_labels):
    """
    Q : Quit (

    0, g = ground (48)
    1, s, m, h = start of multiple, heels (49)
    2, a = air (50)
    n = 9 = no skipper or not skipping
    f = fout
    """
    allowed_keys = ['0', '1', '2', 'n', 'a', 'm', 's', 'h', 'g', 'f']
    quit_key = 'q'
    key_pressed = 'none'
    vid_name, frame_nr, frame = get_random_frame(video_names, video_folder, df_video_border_labels)
    while key_pressed != quit_key:
        while is_already_labeled(vid_name, frame_nr, df_labels):
            vid_name, frame_nr, frame = get_random_frame(video_names, video_folder, df_video_border_labels)
        
        key_pressed = chr(show_frame(frame, 0.5))
        if key_pressed in allowed_keys:
            label = match_label(key_pressed)
            df_labels.loc[len(df_labels)] = [vid_name, frame_nr, label]

    # Close the frame window
    cv2.destroyAllWindows()

In [8]:
df_video_border_labels = pickle_load_or_create(video_border_labels_path, ['path', 'frame', 'border'])
df_video_border_labels.sort_values(['path', 'frame'])

,path,frame,border
370,20240201_atelier_001.mp4,3,9
969,20240201_atelier_001.mp4,8,9
96,20240201_atelier_001.mp4,14,9
298,20240201_atelier_001.mp4,18,9
110,20240201_atelier_001.mp4,28,9
...,...,...,...
779,20240209_atelier_007.mp4,1291,9
686,20240209_atelier_007.mp4,1321,9
975,20240209_atelier_007.mp4,1324,9
609,20240209_atelier_007.mp4,1333,9


In [9]:
label_frames(df_video_border_labels)

../videos/20240209_atelier_006.mp4
../videos/20240201_atelier_005.mp4


../videos/20240201_atelier_003.mp4
../videos/20240201_atelier_002.mp4
../videos/20240209_atelier_007.mp4
../videos/20240201_atelier_004.mp4
../videos/20240201_atelier_002.mp4
../videos/20240201_atelier_002.mp4
../videos/20240201_atelier_004.mp4
../videos/20240201_atelier_005.mp4
../videos/20240201_atelier_003.mp4
../videos/20240201_atelier_001.mp4
../videos/20240201_atelier_004.mp4
../videos/20240201_atelier_003.mp4
../videos/20240201_atelier_001.mp4
../videos/20240201_atelier_003.mp4
../videos/20240201_atelier_002.mp4
../videos/20240201_atelier_001.mp4
../videos/20240201_atelier_002.mp4
../videos/20240209_atelier_006.mp4
../videos/20240209_atelier_006.mp4
../videos/20240201_atelier_001.mp4
../videos/20240201_atelier_004.mp4
../videos/20240201_atelier_002.mp4
../videos/20240209_atelier_006.mp4
../videos/20240209_atelier_006.mp4
../videos/20240209_atelier_006.mp4
../videos/20240201_atelier_003.mp4
../videos/20240201_atelier_003.mp4
../videos/20240209_atelier_006.mp4
../videos/20240201_a

In [10]:
df_video_border_labels

,path,frame,border
0,20240209_atelier_006.mp4,745,0
1,20240209_atelier_007.mp4,103,0
2,20240201_atelier_002.mp4,91,0
3,20240201_atelier_004.mp4,806,2
4,20240201_atelier_003.mp4,622,2
...,...,...,...
1170,20240209_atelier_006.mp4,99,0
1171,20240209_atelier_006.mp4,190,0
1172,20240201_atelier_005.mp4,387,2
1173,20240201_atelier_001.mp4,660,0


In [11]:
with open(video_border_labels_path, 'wb') as handle:
    pickle.dump(df_video_border_labels, handle)

In [12]:
pd.set_option('display.max_rows', None)
df_video_border_labels.sort_values(['path', 'frame'])

,path,frame,border
370,20240201_atelier_001.mp4,3,9
969,20240201_atelier_001.mp4,8,9
96,20240201_atelier_001.mp4,14,9
298,20240201_atelier_001.mp4,18,9
1131,20240201_atelier_001.mp4,24,9
110,20240201_atelier_001.mp4,28,9
616,20240201_atelier_001.mp4,30,9
356,20240201_atelier_001.mp4,35,9
946,20240201_atelier_001.mp4,36,9
555,20240201_atelier_001.mp4,40,9


In [13]:
pd.set_option('display.max_rows', 30)

In [14]:
val_counts = df_video_border_labels.border.value_counts() / len(df_video_border_labels)
val_counts

border
0    0.486809
2    0.245957
9    0.165957
1    0.095319
5    0.005957
Name: count, dtype: float64

In [15]:
val_counts[9] + val_counts[0]

0.6527659574468085